In [1]:
# Import the os package
import os
import shutil

# Import the openai package
import openai
from openai import OpenAI

# From the IPython.display package, import display and Markdown
from IPython.display import display, Markdown

# Use the dotenv package
from dotenv import load_dotenv

# Use the CSV Reader Package
import csv

# Use regular expressions
import re

# Use subprocess
import subprocess

load_dotenv()  # This loads the .env file at the root of the project

# Set openai.api_key to the OPENAI environment variable
client = OpenAI(
    api_key = os.getenv("OPENAI")
)
# print(os.getenv("OPENAI"))

In [2]:
def chat(client, system, user_assistant, openai_model):
    """
    A simple function to handle the mechanics of chatting with ChatGPT
    The model is passed in to make it easy to switch models for different tasks
    """
    assert isinstance(system, str), "`system` should be a string"
    assert isinstance(user_assistant, list), "`user_assistant` should be a list"
    system_msg = [{"role": "system", "content": system}]
    # user_assistant_msgs = [
    #     {"role": "assistant", "content": user_assistant[i]} if i % 2 else {"role": "user", "content": user_assistant[i]}
    #     for i in range(len(user_assistant))]
    user_assistant_msgs = []
    for item in user_assistant:
        content = item['contents']
        msg = {"role": "user", "content": content}
        user_assistant_msgs.append(msg)
    msgs = system_msg + user_assistant_msgs
    response = client.chat.completions.create(model=openai_model, messages=msgs)
    status_code = response.choices[0].finish_reason
    assert status_code == "stop", f"The status code was {status_code}."
    # print(f'{response=}')
    return response.choices[0].message.content

In [3]:
def read_csv_file(file_path):
    """
    Reads a CSV file with each row containing a title and a script path.
    Returns a list of dictionaries with keys 'title' and 'code_path'.
    """
    data = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        data.extend(
            {'title': row[0], 'code_path': row[1]}
            for row in reader
            if len(row) == 2
        )
    return data


In [4]:
def read_files_and_update_csv_data(csv_data, repo_root):
    """
    Reads files specified in the data structure and concatenates their contents.
    
    Parameters:
    - csv_data (list of dict): List of dictionaries with 'title' and 'code_path'.
    - repo_root (str): Repository root base location.

    Returns:
    - str: A single string containing all the titles and file contents.
    """
    csv_data_contents = []
    for item in csv_data:
        full_path = os.path.join(repo_root, item['code_path'])
        # print(f'{full_path=}')
        full_path = full_path.replace(' ./','/')
        try:
            with open(full_path, 'r', encoding='utf-8') as file:
                contents = file.read()
                item['contents'] = contents
                csv_data_contents.append(item)
        except FileNotFoundError:
            concatenated_contents += f"{item['title']}:\n<File not found: {full_path}>\n\n\n"
        except Exception as e:
            concatenated_contents += f"{item['title']}:\n<Error reading file: {e}>\n\n\n"

    return csv_data_contents


In [5]:
def find_repo_root():
    return subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode('utf-8')

In [6]:
def save_markdown_file(item, repo_root, path, response_in):
    title = item['title']
    file_path = os.path.join(repo_root, path, f"{title}.md")

    # Extract the content for each block
    content = response_in

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content.strip() + '\n')

In [8]:
# Read the CSV File
file_path = '/home/sean/github/RTLDesignSherpa/todo_md.csv'
csv_data = read_csv_file(file_path)
print(csv_data)

repo_root = find_repo_root()
print(repo_root)
csv_data_contents = read_files_and_update_csv_data(csv_data, repo_root)
# for item in csv_data_contents:
#     print(f'{item["title"]}:{item["code_path"]}\n{item["contents"]}\n\n\n')

[{'title': 'apb_master_stub', 'code_path': 'rtl/axi/apb_master_stub.sv'}, {'title': 'apb_slave_stub', 'code_path': 'rtl/axi/apb_slave_stub.sv'}, {'title': 'apb_xbar', 'code_path': 'rtl/axi/apb_xbar.sv'}, {'title': 'apb_xbar_thin', 'code_path': 'rtl/axi/apb_xbar_thin.sv'}, {'title': 'axi2apb_convert', 'code_path': 'rtl/axi/axi2apb_convert.sv'}, {'title': 'axi2apb_shim', 'code_path': 'rtl/axi/axi2apb_shim.sv'}, {'title': 'axi_fifo_async', 'code_path': 'rtl/axi/axi_fifo_async.sv'}, {'title': 'axi_fifo_sync', 'code_path': 'rtl/axi/axi_fifo_sync.sv'}, {'title': 'axi_gen_addr', 'code_path': 'rtl/axi/axi_gen_addr.sv'}, {'title': 'axi_master_rd_stub', 'code_path': 'rtl/axi/axi_master_rd_stub.sv'}, {'title': 'axi_master_stub ', 'code_path': 'rtl/axi/axi_master_stub .sv'}, {'title': 'axi_master_wr_stub', 'code_path': 'rtl/axi/axi_master_wr_stub.sv'}, {'title': 'axi_skid_buffer_async', 'code_path': 'rtl/axi/axi_skid_buffer_async.sv'}, {'title': 'axi_skid_buffer', 'code_path': 'rtl/axi/axi_skid_bu

In [9]:
# OpenAI part 1: saving the individual markdown files
openai_model = "gpt-4-1106-preview"
my_header_prompt = '''
In the following text, there will be sections of code that have a header such as:
<block>: <path to block>
followed by a block of code

Please give the Mark Down for the block providing full documentation of the code with knowledge of the other code that is referenced or used as
an include file. The Mark Down for each code block should start with a H1 <block> to indicate when a new block begins.
It should also include full details of the inputs/outputs and internal functionality along with any command line options needed.
At the end of a block, there should be two new lines to provide a visual indicator also.

'''
for item in csv_data_contents:
    prompt = f'{my_header_prompt}{item["title"]}: {item["code_path"]}\n{item["contents"]}\n\n'
    # print(prompt)
    item['contents'] = prompt
    response_fn_test = chat(client, "You are an expert in python and verilog.", [item], openai_model)
    save_markdown_file(item, repo_root, 'docs/mark_down/AxiRTL/', response_fn_test)


In [ ]:
# Code that removes all of the control characters that windows added
def remove_escape_characters(text):
    # List of Markdown special characters
    markdown_chars = ['-', '#', '*', '_', '`', '>', '|', '~', '[', ']', '(', ')']
    
    # Remove backslash before any Markdown special character
    for char in markdown_chars:
        text = text.replace(f"\\{char}", char)
    
    return text

repo_root = find_repo_root()
with open(f'{repo_root}/Combined_MD.txt', 'r') as file:
    text = file.read()

text = remove_escape_characters(text)
# print(text)

In [ ]:
# OpenAI part 2: Fixing the Combined file for consistency
openai_model = "gpt-4-1106-preview"
# prompt = f'''
# The following text is a very long list of markdown files with this demarcation between the blocks
# <block>: <path to block>
# <markdown contents>

# There will also be some index.md and README.md; these shouldn't need much work. Can you leave the contents mostly intact, but do these fixes:
# * I am not looking for a summary of the MD files; I need these fixes below.
# * within script/val/rtl, respectively can you ensure they are strictly consistent
# * across script/val/rtl, respectively can you ensure they are generally consistent
# -- for either case if a block has an inconsistency that you cannot fix add a heading at the every end of the ducument called: ## Inconsistencies and list the block and what is inconsistent that you cannot fix.
# * In many blocks they say they are part of a bigger whole. Can you use the index.md for rtl/scripts to guide you on what is the top block and what are the sub-blocks. Instead of listing that we don't know anything about the other blocks can you use have a hierarchy of links like this:
# [<main block>](<main block>.md)
# * [<sub-block>](<sub-block>.md)
# * etc

# {text}
# '''
prompt = f'''
The following text is a very long list of markdown files with this demarcation between the blocks
<block>: <path to block>
<markdown contents>

There will also be some index.md and README.md; these shouldn't need much work. Can you leave the contents mostly intact, but do these fixes:
* I am not looking for a summary of the MD files; I need these fixes below.
* within script/val/rtl, respectively can you ensure they are strictly consistent
* across script/val/rtl, respectively can you ensure they are generally consistent
-- for either case if a block has an inconsistency that you cannot fix add a heading at the every end of the ducument called: ## Inconsistencies and list the block and what is inconsistent that you cannot fix.

{text}
'''
item = {'contents':prompt}
response_fn_test = chat(client, "You are an expert in python and verilog.", [item], openai_model)

file_path = f'{repo_root}/mark_down_fix.txt'
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(response_fn_test.strip() + '\n')

In [ ]:
sv_code = '''
`timescale 1ns / 1ps

module dataint_ecc_hamming_decode_secded #(
    parameter int WIDTH = 4, parameter int DEBUG = 1
) (
    input  logic                       i_enable,
    input  logic [WIDTH+ParityBits:0]  i_hamming_data,
    output logic [WIDTH-1:0]           ow_data,
    output logic                       ow_error_detected,
    output logic                       ow_double_error_detected
);
    localparam int ParityBits = $clog2(WIDTH + $clog2(WIDTH) + 1);
    localparam int TotalWidth = WIDTH + ParityBits + 1;

    // local wires
    logic [ParityBits:0]   w_syndrome;
    logic [TotalWidth-1:0] w_data_with_parity;
    logic                  w_overall_parity;

    initial begin
        $display("-------------------------------------------");
        $display("Data Width   %d", WIDTH);
        $display("Parity Bits  %d", ParityBits);
        $display("Total Width  %d", TotalWidth);
        $display("-------------------------------------------");
    end

    // Function to calculate the bit position for data extraction
    function automatic integer bit_position(input integer k);
        integer j, pos;
        begin
            pos = k + 1; // Adjust for the SECDED bit
            for (j = 0; j < ParityBits; j = j + 1) begin
                if (pos >= (2**j)) pos = pos + 1;
            end
            bit_position = pos - 1; // Convert to 0-based index
            if (DEBUG)
                $display("Bit position for data bit %d: %d", k, bit_position);
        end
    endfunction

    // Function to get a bit mask for the bits covered by a given parity bit
    function automatic [TotalWidth-1:0] get_covered_bits(input integer parity_bit);
        integer k;
        begin
            get_covered_bits = 0;
            for (k = 0; k < TotalWidth; k = k + 1) begin
                // Check if the k-th bit position is covered by the parity_bit
                if (((k + 1) >> parity_bit) & 1) begin
                    get_covered_bits[k] = 1'b1;
                end
            end
            if (DEBUG)
                $display("Covered bits for parity bit %d: %b", parity_bit, get_covered_bits);
        end
    endfunction

    // Check parity bits
    integer i;
    integer bit_index;
    logic [TotalWidth-1:0] w_covered_bits;
    always_comb begin
        if (i_enable) begin
            for (i = 0; i < ParityBits; i = i + 1) begin
                w_syndrome[i] = 1'b0; // Initialize to 0
                w_covered_bits = get_covered_bits(i);
                for (bit_index = 0; bit_index < TotalWidth; bit_index = bit_index + 1) begin
                    if (w_covered_bits[bit_index]) begin
                        w_syndrome[i] = w_syndrome[i] ^ i_hamming_data[bit_index];
                    end
                end
                if (DEBUG)
                    $display("Syndrome bit %d: %b", i, w_syndrome[i]);
            end
        end else begin
            w_syndrome = 'b0;
        end
    end

    always_comb begin
        if (i_enable) begin
            // Check overall parity
            w_overall_parity = ^i_hamming_data[TotalWidth-2:0];
            if (DEBUG) begin
                $display("i_hamming_data[TotalWidth-2:0]: %b", i_hamming_data[TotalWidth-2:0]);
                $display("Overall parity: %b", w_overall_parity);
            end

            // Calculate syndrome for double error detection
            w_syndrome[ParityBits] = w_overall_parity ^ i_hamming_data[TotalWidth-1];
            if (DEBUG)
                $display("Syndrome for double error detection: %b", w_syndrome[ParityBits]);
        end else begin
            w_overall_parity = 'b0;
            w_syndrome = 'b0;
        end
    end

    // Correct the data if there is a single-bit error
    integer j;
    always_comb begin
        if (i_enable) begin
            $display("w_syndrome: %b", w_syndrome);
            w_data_with_parity = i_hamming_data;
            if (w_syndrome != 0 && w_syndrome != {ParityBits{1'b1}}) begin
                // Single-bit error detected and corrected
                w_data_with_parity[w_syndrome] = ~w_data_with_parity[w_syndrome];
                ow_error_detected = 1'b1;
                ow_double_error_detected = 1'b0;
                $display("Single-bit error detected and corrected at position: %d", w_syndrome);
            end else if (w_syndrome == {ParityBits{1'b1}}) begin
                // Double-bit error detected
                ow_error_detected = 1'b1;
                ow_double_error_detected = 1'b1;
                $display("Double-bit error detected.");
            end else begin
                // No error detected
                ow_error_detected = 1'b0;
                ow_double_error_detected = 1'b0;
                $display("No error detected.");
            end
        end else begin
            w_data_with_parity = 'b0;
            ow_error_detected = 'b0;
            ow_double_error_detected = 'b0;
        end
    end

    // Extract the corrected data
    always_comb begin
        if (i_enable) begin
            for (j = 0; j < WIDTH; j = j + 1) begin
                ow_data[j] = w_data_with_parity[bit_position(j)];
                if (DEBUG)
                    $display("Data bit %d extracted from position %d", j, bit_position(j));
            end
        end else begin
            ow_data = 'b0;
        end

    end

    // synopsys translate_off
    initial begin
        $dumpfile("dump.vcd");
        $dumpvars(0, dataint_ecc_hamming_decode_secded);
    end
    // synopsys translate_on

endmodule : dataint_ecc_hamming_decode_secded

'''

In [ ]:
openai_model = "gpt-4-1106-preview"
repo_root = find_repo_root()
prompt = f'''
The following text is verilog for a hamming decoder. It has an infinite feedack loop in it. Can you identify a couple of possibilities?

{sv_code}
'''
item = {'contents':prompt}
response_fn_test = chat(client, "You are an expert in c, python and verilog.", [item], openai_model)

print(response_fn_test)

# file_path = f'{repo_root}/bch.py'
# with open(file_path, 'w', encoding='utf-8') as file:
#     file.write(response_fn_test.strip() + '\n')

In [ ]:
def split_and_save_md_files(combined_content, repo_root):
    # Regular expression pattern to find sections separated by "Path:"
    pattern = r'Path: [^\n]*'
    paths = re.findall(pattern, combined_content)
    contents = re.split(pattern, combined_content)[1:]  # Ignore the first split which is empty

    if len(paths) != len(contents):
        print("Mismatch between number of paths and contents")
        return

    for path_line, md_content in zip(paths, contents):
        file_path = path_line.replace("Path: ", "").strip()

        if not file_path.startswith(repo_root):
            print(f"Invalid file path: {file_path}")
            continue

        if os.path.exists(file_path):
            old_file_path = f"{file_path}.OLD.md"
            shutil.move(file_path, old_file_path)

        with open(file_path, 'w') as f:
            f.write(md_content.strip() + '\n')


In [ ]:
split_and_save_md_files(text, repo_root)
